# Setup 

In [1]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!apt-get -y update
!apt-get install -y google-chrome-stable
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2023-07-08 06:01:06--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 142.251.31.190, 142.251.31.91, 142.251.31.136, ...
Connecting to dl.google.com (dl.google.com)|142.251.31.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14605 (14K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  14.26K  --.-KB/s    in 0s      

2023-07-08 06:01:06 (110 MB/s) - ‘linux_signing_key.pub’ saved [14605/14605]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1825 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://dl.google.com/linux/chrome/d

In [2]:
# install selenium
!apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

# To check Google Chrome's version
!google-chrome --version

# To check Chrome Driver's version
!chromedriver -v

# Setup Webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)




The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 74 not upgraded.
Need to get 38.7 MB of archives.
After this operation, 174 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.2 [502 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.58+20.04.1 [37.9 MB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64

In [3]:
import io
import os 
import csv
import time
import copy
import urllib
import datetime
import pandas as pd
from tqdm import tqdm
import dateutil.parser
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from joblib import Parallel, delayed
from multiprocessing import cpu_count
import multiprocessing as mp
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

# Scraping Reuter News Articles

#### Sample

In [4]:
# topics = [ "worldNews" ]
# page_number = 1
# page_size = 5

def extract_news( topic, page_size, page_number, display=False ):
    data = pd.DataFrame(columns=["Topic","News Headline", "News Date", "News Summary", "Full News Article", "URL"])
    
    for index in range(page_size):
        driver=webdriver.Chrome(options=chrome_options)
        try:
            driver.get('https://www.reuters.com/news/archive/' + topic + '?view=page&page='+ str(page_number) + '&pageSize='+str(page_size))
            story_content = driver.find_elements_by_class_name("story-content")
            story = story_content[index]
            news_headline = story.text.split("\n")[0]
            news_date =  story.text.split("\n")[2]# datetime.strptime( story.text.split("\n")[2], '%b %d %Y').date()
            news_summary = story.text.split("\n")[1]

            full_news_article = ''
            art = story.find_element_by_xpath('/html/body/div[4]/section[2]/div/div[1]/section/section/div/article['+str(index+1)+']/div[1]/a')
            href = art.get_attribute('href')
            art.click()
            for i in range(2,25):
                try:
                    para = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/main/article/div[1]/div/div/div/div/p['+ str(i) + ']')
                    full_news_article = full_news_article + para.text
                except: break

            data.loc[len(data.index)] = [topic,news_headline,news_date, news_summary, full_news_article, href] 
            driver.quit()

        except:
            print(" Skipping this loop \n ")

        if display: print( "News:", index, "\n News Headline: ", news_headline, "\n", "News Summary: ", news_summary, "\n", "News Date: ", news_date, '\n', "Full News Article: " , full_news_article, "\n", "url: " , href , "\n", " - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -" )
    data.to_csv("reuters_news_"+topic+"_"+str(page_size)+"_"+str(page_number)+".csv") 
    print("reuters_news_"+topic+"_"+str(page_size)+"_"+str(page_number))
    return data

# extract_news( topic=topics[0], page_size=page_size, page_number=page_number, display=Tru  )

In [5]:
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

topics = [ "worldNews", "technologySector", "businessNews" ]
page_size = 10

Number of Logical CPU cores: 4


In [6]:
def download_news_parallel(page_number):
    extract_news(topics[0], page_size, page_number)
# Parallel(n_jobs=n_cores)(delayed(download_news_parallel)(page_number) for page_number in range(1,page_numbers+1) )
pool = mp.Pool(mp.cpu_count())
pool.map(download_news_parallel, [i for i in range(400+1,500+1)])

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
  Skipping this loop 
 

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
reuters_news_worldNews_10_422
 Skipping this loop 
 
reuters_news_worldNews_10_401
 Skipping this loop 
 
 Skipping this loop 
 
reuters_news_worldNe

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [7]:
def download_news_parallel(page_number):
    extract_news(topics[1], page_size, page_number)
# Parallel(n_jobs=n_cores)(delayed(download_news_parallel)(page_number) for page_number in range(1,page_numbers+1) )
pool = mp.Pool(mp.cpu_count())
pool.map(download_news_parallel, [i for i in range(400+1,500+1)])

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
reuters_news_technologySector_10_408
 Skipping this loop 
 
reuters_news_technologySector_10_401
 Skipping this loop 
  Skipping this loop 
 

reuter

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
def download_news_parallel(page_number):
    extract_news(topics[2], page_size, page_number)
# Parallel(n_jobs=n_cores)(delayed(download_news_parallel)(page_number) for page_number in range(1,page_numbers+1) )
pool = mp.Pool(mp.cpu_count())
pool.map(download_news_parallel, [i for i in range(400+1,500+1)])

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
  Skipping this loop 
 

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
  Skipping this loop 
 

 Skipping this loop 
 
 Skipping this loop 
 
 Skipping this loop 
 
reuters_news_businessNews_10_422
 Skipping this loop 
 
reuters_news_businessNews_10_401
 Skipping this loop 
 
reuters_news_businessNews_10_415
 Ski

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Kaggle CPU 96 Cores Multi-processing: https://www.kaggle.com/discussions/general/376783

***